In [26]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


In [27]:
file="50ksample.csv"

In [28]:
data_df=pd.read_csv(file)
data_df.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,1514734,2b7b688e4a4315bc4f21f6849d03ab66,win8defender,1.1.15200.1,4.11.15063.0,1.275.828.0,0,7.0,0,NaN,...,33968.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
1,2926031,53f3fd511ba64acaec4e0c8559143301,win8defender,1.1.15200.1,4.18.1807.18075,1.275.727.0,0,7.0,0,NaN,...,19951.0,0,0.0,0.0,0,0,0.0,1.0,5.0,0
2,7189617,ce48198690d1e4b437ffb60bfcb9e8df,win8defender,1.1.15100.1,4.18.1807.18075,1.273.516.0,0,7.0,0,NaN,...,33066.0,0,NaN,0.0,0,0,0.0,1.0,13.0,1
3,6264912,b3cfee470ee30c729d3a33f0017a8b51,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,...,12303.0,1,NaN,0.0,0,0,0.0,0.0,3.0,0
4,3321923,5f4f1fa2419439ff865d42a2a06ac8b5,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1486.0,0,7.0,0,NaN,...,4357.0,0,NaN,0.0,0,1,0.0,0.0,3.0,1


In [29]:
data_df=data_df[["EngineVersion","AVProductStatesIdentifier","AVProductsEnabled","Processor","SmartScreen","Census_MDC2FormFactor","Census_OSVersion","Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer","HasDetections"]]

data_df.head()


,EngineVersion,AVProductStatesIdentifier,AVProductsEnabled,Processor,SmartScreen,Census_MDC2FormFactor,Census_OSVersion,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,1.1.15200.1,53447.0,1.0,x64,NaN,Desktop,10.0.15063.0,0.0,0.0,0
1,1.1.15200.1,53447.0,1.0,x64,RequireAdmin,Desktop,10.0.16299.125,0.0,1.0,0
2,1.1.15100.1,53447.0,1.0,x64,NaN,Notebook,10.0.15063.1088,0.0,1.0,1
3,1.1.14600.4,47238.0,1.0,x64,NaN,Notebook,10.0.17134.112,0.0,0.0,0
4,1.1.15200.1,53447.0,1.0,x64,NaN,Desktop,10.0.17134.254,0.0,0.0,1


## Iteration 1

Variable choices:



Drop that have nulls in AVProductStatesIdentifier

AVProductsEnabled fill nulls with 0
Browser Identtifier needs to be filled wiht 0
SmartScreen: clean up off and Off, on and On, “&#x02” and “&#x01" will have own columns., null should go to off bucket
Census_IsAlwaysConnectedCapable: replace with 0
Wdft_IsGamer: replace with 0



In [30]:
data_df[["AVProductsEnabled","Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer"]].fillna(value=0, inplace=True)
data_df["SmartScreen"].fillna(value="off", inplace=True)
data_df["SmartScreen"].replace("Off", "off", inplace=True)
data_df["SmartScreen"].replace("On", "on", inplace=True)
data_df["SmartScreen"].fillna(value="off")
## This drops the nulls remaining in AVProductsStatesIdentifier
data_df.dropna(how='any',inplace=True)


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [31]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47734 entries, 0 to 49999
Data columns (total 10 columns):
EngineVersion                              47734 non-null object
AVProductStatesIdentifier                  47734 non-null float64
AVProductsEnabled                          47734 non-null float64
Processor                                  47734 non-null object
SmartScreen                                47734 non-null object
Census_MDC2FormFactor                      47734 non-null object
Census_OSVersion                           47734 non-null object
Census_IsAlwaysOnAlwaysConnectedCapable    47734 non-null float64
Wdft_IsGamer                               47734 non-null float64
HasDetections                              47734 non-null int64
dtypes: float64(4), int64(1), object(5)
memory usage: 4.0+ MB


In [32]:
y=data_df["HasDetections"]
len(y)

47734

In [33]:
X=pd.get_dummies(data_df)
X=X.drop(["HasDetections"],axis=1)
dummy_columns=X.columns.get_values()
dummy_columns=dummy_columns.tolist()

X.head()

,AVProductStatesIdentifier,AVProductsEnabled,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,EngineVersion_1.1.10302.0,EngineVersion_1.1.11701.0,EngineVersion_1.1.12101.0,EngineVersion_1.1.12603.0,EngineVersion_1.1.12805.0,EngineVersion_1.1.12902.0,...,Census_OSVersion_10.0.17746.1000,Census_OSVersion_10.0.17747.1000,Census_OSVersion_10.0.17751.1,Census_OSVersion_10.0.17754.1,Census_OSVersion_10.0.17755.1,Census_OSVersion_10.0.17758.1,Census_OSVersion_10.0.17758.4,Census_OSVersion_10.0.17760.1,Census_OSVersion_10.0.17763.1,Census_OSVersion_10.0.18234.1000
0,53447.0,1.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,53447.0,1.0,0.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,53447.0,1.0,0.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,47238.0,1.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,53447.0,1.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [35]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [36]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )



Training: 0.7793016759776537
Testing: 0.596279537456008


In [37]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, dummy_columns), reverse=True)

[(0.2760334171945992, 'AVProductStatesIdentifier'),
 (0.06652391232671848, 'SmartScreen_ExistsNotSet'),
 (0.03134478109542015, 'Wdft_IsGamer'),
 (0.026049006789877528, 'SmartScreen_RequireAdmin'),
 (0.022022488462731506, 'SmartScreen_off'),
 (0.019769343416605623, 'Census_MDC2FormFactor_Notebook'),
 (0.019063106095887808, 'EngineVersion_1.1.15100.1'),
 (0.01594703921876478, 'Census_MDC2FormFactor_Desktop'),
 (0.0156279563173896, 'EngineVersion_1.1.15200.1'),
 (0.012008058522843772, 'Census_OSVersion_10.0.17134.228'),
 (0.011645597974176788, 'Census_OSVersion_10.0.16299.431'),
 (0.011241322351075105, 'AVProductsEnabled'),
 (0.010510234438628474, 'Census_MDC2FormFactor_Convertible'),
 (0.010091803741432615, 'Census_IsAlwaysOnAlwaysConnectedCapable'),
 (0.010028256382139299, 'Census_OSVersion_10.0.17134.165'),
 (0.00980247675866323, 'Census_OSVersion_10.0.17134.112'),
 (0.009279533628268605, 'Census_MDC2FormFactor_AllInOne'),
 (0.00919535146303341, 'Census_OSVersion_10.0.16299.371'),
 (0.

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(X_test),
                            ))

             precision    recall  f1-score   support

          0       0.60      0.58      0.59      5912
          1       0.60      0.62      0.61      6022

avg / total       0.60      0.60      0.60     11934



## Iteration 2

Some of the previous values did not one hot encode correctly.

Will write a script to encode them and try with that

In [13]:
iter2=data_df.copy()

In [14]:
iter2.head()

,EngineVersion,AVProductStatesIdentifier,AVProductsEnabled,Processor,SmartScreen,Census_MDC2FormFactor,Census_OSVersion,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,1.1.15200.1,53447.0,1.0,x64,off,Desktop,10.0.15063.0,0.0,0.0,0
1,1.1.15200.1,53447.0,1.0,x64,RequireAdmin,Desktop,10.0.16299.125,0.0,1.0,0
2,1.1.15100.1,53447.0,1.0,x64,off,Notebook,10.0.15063.1088,0.0,1.0,1
3,1.1.14600.4,47238.0,1.0,x64,off,Notebook,10.0.17134.112,0.0,0.0,0
4,1.1.15200.1,53447.0,1.0,x64,off,Desktop,10.0.17134.254,0.0,0.0,1


In [15]:
iter2['AVProductStatesIdentifier'] = iter2['AVProductStatesIdentifier'].astype(str)
iter2['AVProductsEnabled'] = iter2['AVProductsEnabled'].astype(str)


In [16]:
X=pd.get_dummies(iter2)
X=X.drop(["HasDetections"],axis=1)
dummy_columns=X.columns.get_values()
dummy_columns=dummy_columns.tolist()

X.head()

,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,EngineVersion_1.1.10302.0,EngineVersion_1.1.11701.0,EngineVersion_1.1.12101.0,EngineVersion_1.1.12603.0,EngineVersion_1.1.12805.0,EngineVersion_1.1.12902.0,EngineVersion_1.1.13000.0,EngineVersion_1.1.13103.0,...,Census_OSVersion_10.0.17746.1000,Census_OSVersion_10.0.17747.1000,Census_OSVersion_10.0.17751.1,Census_OSVersion_10.0.17754.1,Census_OSVersion_10.0.17755.1,Census_OSVersion_10.0.17758.1,Census_OSVersion_10.0.17758.4,Census_OSVersion_10.0.17760.1,Census_OSVersion_10.0.17763.1,Census_OSVersion_10.0.18234.1000
0,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [18]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [19]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )

Training: 0.7793016759776537
Testing: 0.6016423663482487


In [20]:
for i  in range(len(dummy_columns)):
    dummy_columns[i]=str(dummy_columns[i])

In [21]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, dummy_columns), reverse=True)

[(0.06569126229776036, 'SmartScreen_ExistsNotSet'),
 (0.03856815902749786, 'Wdft_IsGamer'),
 (0.03675974407714005, 'AVProductStatesIdentifier_53447.0'),
 (0.029230584415925732, 'SmartScreen_RequireAdmin'),
 (0.02298858847080123, 'SmartScreen_off'),
 (0.021824785345618755, 'Census_MDC2FormFactor_Notebook'),
 (0.021671094267164025, 'EngineVersion_1.1.15100.1'),
 (0.017685015107866485, 'EngineVersion_1.1.15200.1'),
 (0.016802217000621036, 'Census_MDC2FormFactor_Desktop'),
 (0.01393799832146135, 'AVProductStatesIdentifier_7945.0'),
 (0.013141550490260579, 'Census_OSVersion_10.0.17134.228'),
 (0.01308829670000687, 'Census_OSVersion_10.0.16299.431'),
 (0.011433332171471857, 'Census_OSVersion_10.0.17134.165'),
 (0.011345030833288179, 'Census_MDC2FormFactor_Convertible'),
 (0.010391988182100355, 'Census_IsAlwaysOnAlwaysConnectedCapable'),
 (0.010293417373088644, 'Census_OSVersion_10.0.17134.112'),
 (0.009586736314958696, 'Census_OSVersion_10.0.16299.371'),
 (0.009300689199300606, 'Census_MDC2F

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(X_test),
                            ))

             precision    recall  f1-score   support

          0       0.60      0.55      0.58      5912
          1       0.59      0.64      0.61      6022

avg / total       0.60      0.60      0.60     11934



## Iteration 3

We have improvement.

I'm goinig to attempt to just up the estimators

In [39]:
rf = RandomForestClassifier(n_estimators=400)
rf = rf.fit(X_train, y_train)

In [40]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )

Training: 0.7793016759776537
Testing: 0.5935143288084465


In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(X_test),
                            ))

             precision    recall  f1-score   support

          0       0.60      0.55      0.57      5912
          1       0.59      0.63      0.61      6022

avg / total       0.59      0.59      0.59     11934

